In [1]:
import numpy as np
import pandas as pd
import time

import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
import string

#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
names = ['id',
         'headline',
         'description',
         'article_body',
         'date',
         'newspaper',
         'city',
         'md_index',
         'url']

data = pd.read_excel('SV-women-data.xlsx',
                     index_col = 0,
                     names = names)
data

,id,headline,description,article_body,date,newspaper,city,md_index,url
0,1,«Женщины для утех» по-прежнему нужны: Япония и...,Во время колонизации Кореи Японией в XX веке к...,Во время колонизации Кореи Японией в XX веке к...,2016-01-03,ИА Regnum,Москва,2.312,http://regnum.ru/news/2048238.html
1,2,В Норвегии мигрантов научат не насиловать женщин,Наплыв беженцев из Сирии и других стран Ближне...,\n ...,2016-01-04,Известия (iz.ru),Москва,1.113,http://izvestia.ru/news/600956
2,3,Полиция Кельна опровергла сообщения о нападени...,Канцлер ФРГ Ангела Меркель назвала правонаруше...,Об этом сообщил спикер правительства ФРГ Штефф...,2016-01-05,Российская газета (rg.ru),Москва,1.266,http://www.rg.ru/2016/01/05/keln-site.html
3,4,Меркель потребовала дать жесткий ответ на напа...,\tКанцлер Германии Ангела Меркель потребовала ...,"Сейчас+6˚CСейчас в Санкт-ПетербургеОблачно, Бе...",2016-01-05,Фонтанка (fontanka.ru),Санкт-Петербург,1.186,http://www.fontanka.ru/2016/01/05/077/
4,5,В центре Кельна около тысячи мигрантов устроил...,Полиция Германии расследует информацию о серий...,"По словам жертв, они попали в агрессивную толп...",2016-01-06,Комсомольская правда (kp.ru),Москва,0.960,http://www.kp.ru/daily/26476/3347751/
...,...,...,...,...,...,...,...,...,...
15337,15338,Стало известно о гибели в зоне СВО «тулунского...,Заключивший контракт с одной из частных военны...,\n\n\n\n\nЗаключивший контракт с одной из част...,2023-12-28,Свободная пресса (svpressa.ru),Москва,0.922,https://svpressa.ru/war21/news/399920/
15338,15339,Просто эпидемия: старшего брата Майкла Джексон...,"Основателя группы The Jackson 5, старшего брат...","Основателя группы The Jackson 5, старшего брат...",2023-12-29,Eadaily.com,Москва,1.469,https://eadaily.com/ru/news/2023/12/29/prosto-...
15339,15340,Брата Майкла Джексона обвинили в изнасиловании,Брата американского поп-исполнителя Майкла Дже...,\n ...,2023-12-29,Известия (iz.ru),Москва,1.200,https://iz.ru/1628155/2023-12-29/brata-maikla-...
15340,15341,Более 600 деятелей культуры осудили обвиняемог...,Более 600 деятелей культуры осудили обвиняемог...,"30 декабря 2023, 11:49 — Общественная служба...",2023-12-30,Общественная служба новостей (osnmedia.ru),Москва,0.959,https://www.osnmedia.ru/kultura/bolee-600-deya...


In [3]:
data[['headline', 'description', 'article_body']] = data[['headline', 'description', 'article_body']].astype(str)
all_texts = data[['headline', 'description', 'article_body']].agg('-'.join, axis=1)
data.insert(4, 'all_text', all_texts, True)
data

,id,headline,description,article_body,all_text,date,newspaper,city,md_index,url
0,1,«Женщины для утех» по-прежнему нужны: Япония и...,Во время колонизации Кореи Японией в XX веке к...,Во время колонизации Кореи Японией в XX веке к...,«Женщины для утех» по-прежнему нужны: Япония и...,2016-01-03,ИА Regnum,Москва,2.312,http://regnum.ru/news/2048238.html
1,2,В Норвегии мигрантов научат не насиловать женщин,Наплыв беженцев из Сирии и других стран Ближне...,\n ...,В Норвегии мигрантов научат не насиловать женщ...,2016-01-04,Известия (iz.ru),Москва,1.113,http://izvestia.ru/news/600956
2,3,Полиция Кельна опровергла сообщения о нападени...,Канцлер ФРГ Ангела Меркель назвала правонаруше...,Об этом сообщил спикер правительства ФРГ Штефф...,Полиция Кельна опровергла сообщения о нападени...,2016-01-05,Российская газета (rg.ru),Москва,1.266,http://www.rg.ru/2016/01/05/keln-site.html
3,4,Меркель потребовала дать жесткий ответ на напа...,\tКанцлер Германии Ангела Меркель потребовала ...,"Сейчас+6˚CСейчас в Санкт-ПетербургеОблачно, Бе...",Меркель потребовала дать жесткий ответ на напа...,2016-01-05,Фонтанка (fontanka.ru),Санкт-Петербург,1.186,http://www.fontanka.ru/2016/01/05/077/
4,5,В центре Кельна около тысячи мигрантов устроил...,Полиция Германии расследует информацию о серий...,"По словам жертв, они попали в агрессивную толп...",В центре Кельна около тысячи мигрантов устроил...,2016-01-06,Комсомольская правда (kp.ru),Москва,0.960,http://www.kp.ru/daily/26476/3347751/
...,...,...,...,...,...,...,...,...,...,...
15337,15338,Стало известно о гибели в зоне СВО «тулунского...,Заключивший контракт с одной из частных военны...,\n\n\n\n\nЗаключивший контракт с одной из част...,Стало известно о гибели в зоне СВО «тулунского...,2023-12-28,Свободная пресса (svpressa.ru),Москва,0.922,https://svpressa.ru/war21/news/399920/
15338,15339,Просто эпидемия: старшего брата Майкла Джексон...,"Основателя группы The Jackson 5, старшего брат...","Основателя группы The Jackson 5, старшего брат...",Просто эпидемия: старшего брата Майкла Джексон...,2023-12-29,Eadaily.com,Москва,1.469,https://eadaily.com/ru/news/2023/12/29/prosto-...
15339,15340,Брата Майкла Джексона обвинили в изнасиловании,Брата американского поп-исполнителя Майкла Дже...,\n ...,Брата Майкла Джексона обвинили в изнасиловании...,2023-12-29,Известия (iz.ru),Москва,1.200,https://iz.ru/1628155/2023-12-29/brata-maikla-...
15340,15341,Более 600 деятелей культуры осудили обвиняемог...,Более 600 деятелей культуры осудили обвиняемог...,"30 декабря 2023, 11:49 — Общественная служба...",Более 600 деятелей культуры осудили обвиняемог...,2023-12-30,Общественная служба новостей (osnmedia.ru),Москва,0.959,https://www.osnmedia.ru/kultura/bolee-600-deya...


In [4]:
def preprocess(text, punc, analyzer, stopw, alph):
    
    tokens = word_tokenize(text.replace('.', ' ').lower())
    tokens = [token for token in tokens if token not in punc]
    tokens = [analyzer.parse(token)[0].normal_form for token in tokens]
    tokens = [token for token in tokens if token not in stopw]
    tokens = [token for token in tokens if token.isalpha() == True]
    
    for token in tokens:
        for letter in token:
            if letter not in alph:
                tokens.remove(token)
                break
                
    return(tokens)

In [5]:
punctuation = open("punctuation.txt", "r", encoding = 'utf-8').read().strip().replace('\n', '') + string.punctuation 
morph = pymorphy3.MorphAnalyzer()
#stop_words = stopwords.words('russian') # слишком мало слов
stop_words = open("stopwords-ru.txt", "r", encoding = 'utf-8').read().strip().split('\n')
alphabet = string.ascii_uppercase + \
           string.ascii_lowercase + \
           'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [6]:
idx = np.random.randint(data.shape[0], size=1)[0]

preprocess(data.all_text[idx],
           punctuation,
           morph,
           stop_words, 
           alphabet)

['глава',
 'мид',
 'фрг',
 'анджелина',
 'джоля',
 'объединиться',
 'борьба',
 'сексуальный',
 'насилие',
 'кризисный',
 'зона',
 'стать',
 'главный',
 'председательство',
 'германия',
 'совет',
 'безопасность',
 'организация',
 'объединить',
 'апрель',
 'иа',
 'регнум',
 'борьба',
 'сексуальный',
 'насилие',
 'кризисный',
 'зона',
 'стать',
 'главный',
 'председательство',
 'германия',
 'совет',
 'безопасность',
 'организация',
 'объединить',
 'нация',
 'оон',
 'голливудский',
 'актриса',
 'анджелина',
 'джоля',
 'поддержать',
 'усилие',
 'берлин',
 'опубликовать',
 'совместный',
 'гостевой',
 'статья',
 'министр',
 'иностранный',
 'дело',
 'германия',
 'хайко',
 'маас',
 'газета',
 'the',
 'washington',
 'post',
 'автор',
 'статья',
 'выступать',
 'поддержка',
 'жертва',
 'насилие',
 'улучшение',
 'механизм',
 'судебный',
 'преследование',
 'насильник',
 'оба',
 'приводить',
 'пример',
 'сложный',
 'судьба',
 'женщина',
 'кризисный',
 'зона',
 'изнасилование',
 'форма',
 'сексуальный

In [7]:
%%time

texts_lem = []

for text in data.all_text:
    text_lem = preprocess(text, punctuation, morph, stop_words, alphabet)
    text_lem_str = " ".join(str(word) for word in text_lem)
    texts_lem.append(text_lem_str)

CPU times: total: 22min 57s
Wall time: 23min 35s


In [8]:
data.insert(5, 'text_lem', texts_lem, True)
data

,id,headline,description,article_body,all_text,text_lem,date,newspaper,city,md_index,url
0,1,«Женщины для утех» по-прежнему нужны: Япония и...,Во время колонизации Кореи Японией в XX веке к...,Во время колонизации Кореи Японией в XX веке к...,«Женщины для утех» по-прежнему нужны: Япония и...,женщина утеха нужный япония китай борьба время...,2016-01-03,ИА Regnum,Москва,2.312,http://regnum.ru/news/2048238.html
1,2,В Норвегии мигрантов научат не насиловать женщин,Наплыв беженцев из Сирии и других стран Ближне...,\n ...,В Норвегии мигрантов научат не насиловать женщ...,норвегия мигрант научить насиловать беженец си...,2016-01-04,Известия (iz.ru),Москва,1.113,http://izvestia.ru/news/600956
2,3,Полиция Кельна опровергла сообщения о нападени...,Канцлер ФРГ Ангела Меркель назвала правонаруше...,Об этом сообщил спикер правительства ФРГ Штефф...,Полиция Кельна опровергла сообщения о нападени...,полиция кёльн опровергнуть сообщение нападение...,2016-01-05,Российская газета (rg.ru),Москва,1.266,http://www.rg.ru/2016/01/05/keln-site.html
3,4,Меркель потребовала дать жесткий ответ на напа...,\tКанцлер Германии Ангела Меркель потребовала ...,"Сейчас+6˚CСейчас в Санкт-ПетербургеОблачно, Бе...",Меркель потребовала дать жесткий ответ на напа...,меркель потребовать дать жёсткий ответ нападен...,2016-01-05,Фонтанка (fontanka.ru),Санкт-Петербург,1.186,http://www.fontanka.ru/2016/01/05/077/
4,5,В центре Кельна около тысячи мигрантов устроил...,Полиция Германии расследует информацию о серий...,"По словам жертв, они попали в агрессивную толп...",В центре Кельна около тысячи мигрантов устроил...,центр кёльн тысяча мигрант устроить новогодний...,2016-01-06,Комсомольская правда (kp.ru),Москва,0.960,http://www.kp.ru/daily/26476/3347751/
...,...,...,...,...,...,...,...,...,...,...,...
15337,15338,Стало известно о гибели в зоне СВО «тулунского...,Заключивший контракт с одной из частных военны...,\n\n\n\n\nЗаключивший контракт с одной из част...,Стало известно о гибели в зоне СВО «тулунского...,стать известно гибель зона сво тулунский манья...,2023-12-28,Свободная пресса (svpressa.ru),Москва,0.922,https://svpressa.ru/war21/news/399920/
15338,15339,Просто эпидемия: старшего брата Майкла Джексон...,"Основателя группы The Jackson 5, старшего брат...","Основателя группы The Jackson 5, старшего брат...",Просто эпидемия: старшего брата Майкла Джексон...,просто эпидемия старший брат майкл джексон обв...,2023-12-29,Eadaily.com,Москва,1.469,https://eadaily.com/ru/news/2023/12/29/prosto-...
15339,15340,Брата Майкла Джексона обвинили в изнасиловании,Брата американского поп-исполнителя Майкла Дже...,\n ...,Брата Майкла Джексона обвинили в изнасиловании...,брат майкл джексон обвинить американский майкл...,2023-12-29,Известия (iz.ru),Москва,1.200,https://iz.ru/1628155/2023-12-29/brata-maikla-...
15340,15341,Более 600 деятелей культуры осудили обвиняемог...,Более 600 деятелей культуры осудили обвиняемог...,"30 декабря 2023, 11:49 — Общественная служба...",Более 600 деятелей культуры осудили обвиняемог...,деятель культура осудить обвинять домогательст...,2023-12-30,Общественная служба новостей (osnmedia.ru),Москва,0.959,https://www.osnmedia.ru/kultura/bolee-600-deya...


In [9]:
data.to_excel('SV-women-data-lem.xlsx')